In [2]:
!pip3 install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB ? eta 0:00:00B/s eta 0:00:010:00:21
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 709.6 kB/s eta 0:00:00m eta 0:00:010:00:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━ 378.3/557.1 MB 1.4 MB/s eta 0:02:09:18

In [1]:
from multi_pure_prep import init_state_exp_val
from multi_pure_prep import train,gen_paulis
from multi_pure_prep import device_and_random_params
from multi_pure_prep import cost,circuit
import pennylane as qml


ModuleNotFoundError: No module named 'torch'

In [10]:
target_vector = init_state_exp_val(2)
device, params = device_and_random_params()
qc = circuit(params, target_vector)
#qnode = qml.QNode(qc, device=device, interface="torch")
#qnode = qml.QNode(circuit, device=device, interface="torch")
best_params = train(30, params)

TypeError: 'MeasurementProcess' object is not callable

In [ ]:
epochs, fidelity = train(30, circuit, params, cost(params, Paulis, target_vector))

In [ ]:
target_op, params = init_state_params()
best_params, best_loss = train(params,target_op)